# These are routines to put in physically motivated profiles --- either that match the X-ray data or simulations

In [4]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.interpolate import interp1d
import scipy.special as sc
import scipy.integrate as integrate


In [5]:
#fundamental constants
mp = 1.6726216e-24  #proton mass in gr
# KPCTOCM = 3.086e21
MPCTOCM = 3.086e24
Msun =  1.9889e33  # gr 

#cosmology "constants"
Omegab = 0.045
OmegaM=0.3 
fb = Omegab/OmegaM #fraction of matter in baryons
h = 0.7
rhocrit = 2.7755e11*h*h
YHE = 0.25 #helium mass fraction
mu = 1/((1-YHE) + 2.*YHE/4)  #mean molecular weight to give number of electrons assuming helium doubly ionized


In [6]:
#from memory.  Adnan, substitute your expression 
# def rvir(M, z):
#     return 280*(M/1e12)**(.33)/(1+z) #kpc

# def rvir(M, z):
#     return .001*280*(M/1e12)**(.33)/(1+z) #Mpc


#radius that is 200 times the matter density in Mpc  (very similar to rvir)
def r200Mz(Mhalo, z):
    rhomatter = rhocrit*OmegaM
    return (Mhalo/((4*np.pi/3)*200.*rhomatter))**(1./3.);

# I'm not sure I'm using this
def rhoNFW(r, rs):
    return 1/(r/rs)/(1+(r/rs))**2

 # This is FIRE simulation profile (from eyeballing https://arxiv.org/pdf/1811.11753.pdf and using their $r^{-2}$ scaling)

 I'm using the $r^{-2}$ profile they find, with an exponetial cutoff at rmax, where we use conservation of mass to determine rmax.  So
 
 $$\rho = \rho_{0}\left(\frac{r}{r_*} \right)^{-2} \exp[-r/r_{\rm max}]$$
 
 They results technically hold for $10^{10} - 10^{12} M_\odot$ and $z=0$, but I think it's reasonable to assume they extrapolate to other moderate redshifts and somewhat more massive halos.
 
 To normalize things we are using that 
 $$M_{\rm gas} = \int 4 \pi r^2 dr \rho_{0} \left(\frac{r}{r_*} \right)^{-2} \exp[-r/r_{\rm max}] = 4 \pi r_{\rm max} \rho_0 r_*^2$$
 
 Note that sometimes I'm working with number densities and sometimes mass densities

In [1]:
adjustmentfactor = 1  #probably we want to range between 0.5-2 as the numbers look sensible in this range


#These are specifications taken from the fire simulations
RinterpinRvir = 0.3  # this is the point where I read off the density nrmalization
logMinterp = np.array([10., 11., 12.])  # these are log10 of the halo masses they consider
nHinterp = np.array([0.5e-4, 0.8e-4, 1e-4])  #these are their number densities in cubic cm



    
nHinterp = interp1d(logMinterp, nHinterp, fill_value="extrapolate")


#The density of 
def rhogasFire(Mhalo,Rvir, z, adjustmentfactor,res):
    
    # Simulation details
    
    h=.7
    L=250/h #Mpc
    resolution=1024*res
    cellsize = L/resolution
    y,x = np.ogrid[-20*res: 20*res, -20*res:20*res]


    
    
    
    rho0 = adjustmentfactor*nHinterp(np.log10(Mhalo))
    Rinterp = RinterpinRvir* Rvir #rvir(Mhalo, z)
    
    
#     Rinterp = RinterpinRvir*rvir(Mhalo, z)
    
#     Ntot = Mhalo*Msun*fb/(mu*mp)/(KPCTOCM**3)
    Ntot = Mhalo*Msun*fb/(mu*mp)/(MPCTOCM**3)
    rmax = Ntot/(4.*np.pi*rho0*Rinterp**2 )  #from integrating above expression for rho
#     print("rho0 = ", rho0, "rmax = ", rmax, "rmax/rvir = ", rmax/rvir(Mhalo, z), "[don't want last to be much smaller than virial radius]")
    
    
    rarr = np.logspace(-2, np.log10(5*rmax), 100) # Cut off at 5 times exponential cutoff
    rhoarr = rho0*(rarr/Rinterp)**-2*np.exp(-rarr/rmax) #number density: per cm3
    #rhoarr[rarr > rmax] = 0
    
    
    ## creating a mask
    f1= lambda x, y, z: my_func(((x**2+y**2+z**2)**.5), rmax,Rinterp,rho0,cellsize)


    
    vec_integral=np.vectorize(func3Dto2D)
    
    

    mask1 = vec_integral(f1,x,y,rmax/cellsize)
#     mask1 = vec_integral(f1,x,y,Rvir/cellsize)
    r=(x**2+y**2)**.5 # * scale_down
    mask1=mask1.astype(float)
#     mask1[r > (rmax/cellsize)]=0        
    
    
    return [rarr, rhoarr,mask1]
    
    

NameError: name 'np' is not defined

In [31]:
# The user can create their own function
# All length scales have to be converted into units of cellsize
def my_func(r, rmax,Rinterp,rho0,cellsize):
    R1 = rmax/cellsize  # Convert length scale into units of cellsize
    R2 = Rinterp/cellsize
    return rho0*np.exp(-r/R1)*  ((r+.5)/R2)**-2
           


In [32]:
def radial_profile(data, center_x,center_y):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

In [33]:
def func3Dto2D(f,x,y,Rvir):
    
    return integrate.quad(f,-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0] 
    


rm10 = rhogasFire(1e14, .2, adjustmentfactor,1,1)
# rm40 = rhogasFire(1e10, 0, adjustmentfactor,2)

plt.imshow(rm10[2])

plt.imshow(rm10[2])

plt.imshow(rm10[2])

rm10 = rhogasFire(1e10, 0, adjustmentfactor,1)

np.max(rm10[2])

np.max(rm40[2])

plt.imshow(rm10[2],vmin=np.min(rm10[2]),vmax=np.max(rm10[2]))
np.mean(rm10[2])
plt.show()

plt.imshow(rm40[2],vmin=np.min(rm40[2]),vmax=np.max(rm40[2]))
np.mean(rm40[2])
plt.show()


rad10=radial_profile(rm10[2], 5,5)
rad40=radial_profile(rm40[2], 10,10)

plt.plot(rad10)
plt.plot(rad40)

# Test: General case of analytic function

%run FRBProject-Chap1-ParametersAndFunctions.ipynb
%run FRBProject-Chap2-ExtractData-Halos.ipynb
%run FRBProject-Chap3-Convolution.ipynb


current_halo_file = extract_halos(0)

resolution = 1



min_mass=10**12
max_mass=10**14
log_bins=10

subtraction_halo_profile = 'NFW'
addition_halo_profile = 'tophat' 
scaling_radius = 1
redshift=0


halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
df= halo_array_for_convolution[0]
binz= halo_array_for_convolution[1]
    

    
    
    # convolve halos
profile = add_halos(df,resolution,len(binz)-1,binz,subtraction_halo_profile,scaling_radius,redshift)

plt.imshow(profile[4])
print(np.mean(profile[0]))

resolution = 2



min_mass=10**12
max_mass=10**14
log_bins=10

subtraction_halo_profile = 'NFW'
addition_halo_profile = 'tophat' 
scaling_radius = 1
redshift=0


halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
df= halo_array_for_convolution[0]
binz= halo_array_for_convolution[1]
    

    
    
    # convolve halos
profile1_2 = add_halos(df,resolution,len(binz)-1,binz,subtraction_halo_profile,scaling_radius,redshift)

plt.imshow(profile1_2[4])
print(np.mean(profile1_2[0]))

resolution = 1

current_halo_file = extract_halos(0)

min_mass=10**12
max_mass=10**14
log_bins=10

subtraction_halo_profile = 'NFW'
addition_halo_profile = 'fire' 
scaling_radius = 1
redshift=0


halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
df= halo_array_for_convolution[0]
binz= halo_array_for_convolution[1]
    

    
    
    # convolve halos
profile_fire = add_halos_test(df,resolution,len(binz)-1,binz,subtraction_halo_profile,scaling_radius,redshift)

plt.imshow(profile_fire[4])
print(np.mean(profile_fire[0]))

resolution = 2


min_mass=10**12
max_mass=10**14
log_bins=10

subtraction_halo_profile = 'NFW'
addition_halo_profile = 'fire' 
scaling_radius = 1
redshift=0


halo_array_for_convolution = create_halo_array_for_convolution(current_halo_file,min_mass,max_mass,log_bins)
df= halo_array_for_convolution[0]
binz= halo_array_for_convolution[1]
    

    
    
    # convolve halos
profile_fire_2 = add_halos_test(df,resolution,len(binz)-1,binz,subtraction_halo_profile,scaling_radius,redshift)

plt.imshow(profile_fire_2[4])
print(np.mean(profile_fire_2[0]))